In [25]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections
from pyspark.ml.linalg import Vectors
from sklearn.cluster import MiniBatchKMeans, KMeans
import numpy as np

In [26]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "C:/temp").appName("SparkSQL").getOrCreate()

In [27]:
inputLines = spark.sparkContext.textFile("test1.csv")
#inputLines = spark.sparkContext.textFile("train_full_parsed_clean5.1.csv")

In [28]:
header=inputLines.filter(lambda l: "_id" in l)

In [48]:
header.collect()

['log_duration,log_trip_duration,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,distance,speed_great_circle,hour_count,snow,holiday,vendor_id,pickup_hour_0,pickup_hour_1,pickup_hour_2,pickup_hour_3,pickup_hour_4,pickup_hour_5,pickup_hour_6,pickup_hour_7,pickup_hour_8,pickup_hour_9,pickup_hour_10,pickup_hour_11,pickup_hour_12,pickup_hour_13,pickup_hour_14,pickup_hour_15,pickup_hour_16,pickup_hour_17,pickup_hour_18,pickup_hour_19,pickup_hour_20,pickup_hour_21,pickup_hour_22,pickup_hour_23,pickup_weekday_0,pickup_weekday_1,pickup_weekday_2,pickup_weekday_3,pickup_weekday_4,pickup_weekday_5,pickup_weekday_6,pickup_month_1,pickup_month_2,pickup_month_3,pickup_month_4,pickup_month_5,pickup_month_6,passenger_count_0,passenger_count_1,passenger_count_2,passenger_count_3,passenger_count_4,passenger_count_5,passenger_count_6,passenger_count_7,route_cooridnates']

In [5]:
linesnoheader = inputLines.subtract(header)

In [50]:
linesnoheader.take(2)

['6.046426105181958,6.1224928095143865,-73.98215484619139,40.76793670654297,-73.96463012695312,40.765602111816406,1498.9429809955814,2008.2,11.85976864084416,1769,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,"[[-73.982316, 40.767869], [-73.982163, 40.767742], [-73.981921, 40.76768], [-73.981659, 40.767725], [-73.981479, 40.767854], [-73.981407, 40.768071], [-73.981552, 40.768428], [-73.981523, 40.768582], [-73.979193, 40.771782], [-73.978784, 40.771698], [-73.977952, 40.771794], [-73.977665, 40.771708], [-73.97391, 40.769391], [-73.973406, 40.769184], [-73.971555, 40.768635], [-73.970281, 40.768025], [-73.964608, 40.765631]]"',
 '6.376726947898628,6.498282149476434,-73.98041534423827,40.738563537597656,-73.99948120117188,40.73115158081055,1806.0158621526614,2514.5,9.80642097096468,3912,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,"[[-73.980421, 40.738566], [-73.984941, 40.732347], [-73

In [29]:
coord=pd.read_csv('newcounter.csv')
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000,random_state=1234).fit(coord[['longitude','latitude']])
coord.loc[:, 'cluster'] = kmeans.predict(coord[['longitude','latitude']])

speed_cluster=pd.read_csv("hour_speed_cluster.csv")

In [30]:
cluster=coord['cluster'].tolist()

cluster_name=['C'+str(i) for i in range(100)]


In [31]:
def topcoordinates(line):
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)

    line_cluster=[]
    for i in coordinates:
                temp=re.split('\[|, |\]',i)
                long=float(temp[1])
                lat=float(temp[2])
                cluster_group=kmeans.predict([[long,lat]])
                line_cluster+=[cluster_group[0]]

    line_cluster=list(set(line_cluster))
    feature=line.split(',')
    all_cluster=[ float(speed_cluster.loc[speed_cluster.hour_count==int(feature[9]) , 'C'+str(i)].values[0]) if i in line_cluster else np.nan for i in range(100)]





    feature1=[float(i) for i in feature[:58]]
    feature2=[int(i) for i in feature1[10:58]]
    feature3=feature1[:8]
    return feature3+feature2+all_cluster
    

In [63]:
line='6.376726947898628,6.498282149476434,-73.98041534423827,40.738563537597656,-73.99948120117188,40.73115158081055,1806.0158621526614,2514.5,9.80642097096468,3912,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,"[[-73.980421, 40.738566], [-73.984941, 40.732347], [-73.993607, 40.736013], [-73.996976, 40.731414], [-73.998612, 40.732215], [-73.999502, 40.731162]]"'
coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    
line_cluster=[]
for i in coordinates:
            temp=re.split('\[|, |\]',i)
            long=float(temp[1])
            lat=float(temp[2])
            cluster_group=kmeans.predict([[long,lat]])
            line_cluster+=[cluster_group[0]]

line_cluster=list(set(line_cluster))
feature=line.split(',')
all_cluster=[ float(speed_cluster.loc[speed_cluster.hour_count==int(feature[9]) , 'C'+str(i)].values[0]) if i in line_cluster else np.nan for i in range(100)]
        
    
    
    

feature1=[float(i) for i in feature[:58]]
feature2=[int(i) for i in feature1[10:58]]
feature3=feature1[:8]

In [69]:
feature3

[6.376726947898628,
 6.498282149476434,
 -73.98041534423827,
 40.738563537597656,
 -73.99948120117188,
 40.73115158081055,
 1806.0158621526614,
 2514.5]

In [32]:
coord = linesnoheader.map(topcoordinates)

In [33]:
feature=['log_duration', 'log_trip_duration', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'great_circle_distance','distance',
       'snow', 'holiday', 'vendor_id', 'pickup_hour_0',
       'pickup_hour_1', 'pickup_hour_2', 'pickup_hour_3', 'pickup_hour_4',
       'pickup_hour_5', 'pickup_hour_6', 'pickup_hour_7', 'pickup_hour_8',
       'pickup_hour_9', 'pickup_hour_10', 'pickup_hour_11',
       'pickup_hour_12', 'pickup_hour_13', 'pickup_hour_14',
       'pickup_hour_15', 'pickup_hour_16', 'pickup_hour_17',
       'pickup_hour_18', 'pickup_hour_19', 'pickup_hour_20',
       'pickup_hour_21', 'pickup_hour_22', 'pickup_hour_23',
       'pickup_weekday_0', 'pickup_weekday_1', 'pickup_weekday_2',
       'pickup_weekday_3', 'pickup_weekday_4', 'pickup_weekday_5',
       'pickup_weekday_6', 'pickup_month_1', 'pickup_month_2',
       'pickup_month_3', 'pickup_month_4', 'pickup_month_5',
       'pickup_month_6', 'passenger_count_0', 'passenger_count_1',
       'passenger_count_2', 'passenger_count_3', 'passenger_count_4',
       'passenger_count_5', 'passenger_count_6', 'passenger_count_7']
colNames=feature+cluster_name

In [39]:
colNames

['log_duration',
 'log_trip_duration',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'great_circle_distance',
 'distance',
 'snow',
 'holiday',
 'vendor_id',
 'pickup_hour_0',
 'pickup_hour_1',
 'pickup_hour_2',
 'pickup_hour_3',
 'pickup_hour_4',
 'pickup_hour_5',
 'pickup_hour_6',
 'pickup_hour_7',
 'pickup_hour_8',
 'pickup_hour_9',
 'pickup_hour_10',
 'pickup_hour_11',
 'pickup_hour_12',
 'pickup_hour_13',
 'pickup_hour_14',
 'pickup_hour_15',
 'pickup_hour_16',
 'pickup_hour_17',
 'pickup_hour_18',
 'pickup_hour_19',
 'pickup_hour_20',
 'pickup_hour_21',
 'pickup_hour_22',
 'pickup_hour_23',
 'pickup_weekday_0',
 'pickup_weekday_1',
 'pickup_weekday_2',
 'pickup_weekday_3',
 'pickup_weekday_4',
 'pickup_weekday_5',
 'pickup_weekday_6',
 'pickup_month_1',
 'pickup_month_2',
 'pickup_month_3',
 'pickup_month_4',
 'pickup_month_5',
 'pickup_month_6',
 'passenger_count_0',
 'passenger_count_1',
 'passenger_count_2',
 'passenger_count_3',
 'passen

In [24]:
train = coord.toDF(colNames)

KeyboardInterrupt: 

In [ ]:
train.write.csv("train_full_parsed_clean5.csv",header = 'true')

In [ ]:
train.repartition(1).write.format('com.databricks.spark.csv').save('csv_cluster.csv',header = 'true')

In [73]:
train.repartition(1).write.parquet('parquet_cluster_40_files_v2')

In [35]:
df=pd.read_csv("train_full_parsed_clean5.1.csv")

In [75]:
#df=df.loc[:100,]

In [34]:
#df.loc[:1000,].to_csv("test1.csv",index=False)

In [41]:
i=0
def finderror(start,end,df,colNames,topcoordinates):
    global i;
    print("start: "+str(start)+" end: "+str(end))
    df.loc[start:end,].to_csv("test1.csv",index=False)
    inputLines = spark.sparkContext.textFile("test1.csv")
    header=inputLines.filter(lambda l: "_id" in l)
    linesnoheader = inputLines.subtract(header)
    try:
        coord = linesnoheader.map(topcoordinates)
        train = coord.toDF(colNames)
    
        train.repartition(1).write.csv(str(i),header='true')
        print("good")
        i+=1
        return 0
    except:
        print("error in "+str(start)+" to "+str(end))
        i+=1
        return 1


In [42]:
start=0
end=df.shape[0]
end=20

mid=729321
mid=10

while True:
    result=finderror(start,mid,df,colNames,topcoordinates)
    if result==1:
        end=mid;
        mid=int(np.floor((start+mid)/2))
    elif result==0:
        start=mid;
        mid=int(np.floor((end+mid)/2))
    if mid-start==0:
        print("end: "+str(mid)+" start "+str(start)+" so breaks")
        break

start: 0 end: 10
good
start: 10 end: 15
good
start: 15 end: 17
good
start: 17 end: 18
good
start: 18 end: 19
good
end: 19 start 19 so breaks


In [20]:
a[0:1]

[1]

In [ ]:
df

,log_duration,log_trip_duration,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,great_circle_distance,distance,speed_great_circle,hour_count,...,pickup_month_6,passenger_count_0,passenger_count_1,passenger_count_2,passenger_count_3,passenger_count_4,passenger_count_5,passenger_count_6,passenger_count_7,route_cooridnates
0,6.046426,6.122493,-73.982155,40.767937,-73.964630,40.765602,1498.942981,2008.2,11.859769,1769,...,0,0,1,0,0,0,0,0,0,"[[-73.982316, 40.767869], [-73.982163, 40.7677..."
1,6.376727,6.498282,-73.980415,40.738564,-73.999481,40.731152,1806.015862,2514.5,9.806421,3912,...,1,0,1,0,0,0,0,0,0,"[[-73.980421, 40.738566], [-73.984941, 40.7323..."
2,7.156177,7.661527,-73.979027,40.763939,-74.005333,40.710087,6386.897467,11060.5,10.825250,443,...,0,0,1,0,0,0,0,0,0,"[[-73.978874, 40.764148], [-73.977685, 40.7636..."
3,6.137079,6.063785,-74.010040,40.719971,-74.012268,40.706718,1485.916955,1819.5,12.469233,2323,...,0,0,1,0,0,0,0,0,0,"[[-74.010145, 40.719982], [-74.011102, 40.7155..."
4,5.861925,6.077642,-73.973053,40.793209,-73.972923,40.782520,1188.923339,1614.9,9.839366,2053,...,0,0,1,0,0,0,0,0,0,"[[-73.972998, 40.793187], [-73.976607, 40.7883..."
5,6.081305,6.095825,-73.982857,40.742195,-73.992081,40.749184,1099.252081,1395.4,8.932974,718,...,0,0,0,0,0,0,0,1,0,"[[-73.982805, 40.742173], [-73.982914, 40.7420..."
6,5.922114,5.834811,-73.969017,40.757839,-73.957405,40.765896,1326.652250,1703.2,14.005713,4054,...,1,0,0,0,0,1,0,0,0,"[[-73.969209, 40.757919], [-73.963843, 40.7653..."
7,7.410831,7.347300,-73.969276,40.797779,-73.922470,40.760559,5716.590800,19652.8,13.268683,3391,...,0,0,1,0,0,0,0,0,0,"[[-73.96929, 40.79776], [-73.971919, 40.79471]..."
8,5.768633,5.545177,-73.999481,40.738400,-73.985786,40.732815,1310.722469,1310.7,18.504317,3551,...,0,0,1,0,0,0,0,0,0,"[[-73.999444, 40.738451], [-73.985848, 40.73273]]"
9,7.368592,7.111512,-73.981049,40.744339,-73.973000,40.789989,5122.604426,11045.6,15.054184,1677,...,0,0,1,0,0,0,0,0,0,"[[-73.981184, 40.744396], [-73.981561, 40.7438..."


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 54982)
----------------------------------------


Traceback (most recent call last):
  File "C:\Users\TANGP05\AppData\Local\Continuum\Anaconda3\lib\socketserver.py", line 313, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\TANGP05\AppData\Local\Continuum\Anaconda3\lib\socketserver.py", line 341, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\TANGP05\AppData\Local\Continuum\Anaconda3\lib\socketserver.py", line 354, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "C:\Users\TANGP05\AppData\Local\Continuum\Anaconda3\lib\socketserver.py", line 681, in __init__
    self.handle()
  File "c:/spark\python\pyspark\accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "c:/spark\python\pyspark\serializers.py", line 579, in read_int
    length = stream.read(4)
  File "C:\Users\TANGP05\AppData\Local\Continuum\Anaconda3\lib\socket.py", line 575, in readinto
    return self._sock.recv_into(b)
Connecti